Autor: Raúl Jiménez Juárez
Fecha: 08/11/2019

Notebook principal donde se ejecuta el optimizador

In [ ]:
#Librerías importadas
import datetime
import sys
sys.path.insert(0, '/')
import modulos.optimizador
from modulos.optimizador import HVAcOptimizer
from modulos.datos_externos import DatosExternos
from modulos.visualizacion import MostrarFrente
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing
#%matplotlib inline

In [ ]:
#Función para calcular el fitness
def calcular_fitness(valores_objetivos):
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(abs(valores_objetivos))
    
    #cofort[0], consumo[1], COP[2], vt[3], tshow[4]
    fitness_individuos = []
    for solucion in x_scaled:
        fitness = abs(solucion[0])*10  + solucion[1]*5 - solucion[2]*(1) + abs(solucion[3]) + solucion[4]
        fitness_individuos.append(fitness)
    return fitness_individuos

## Configuración para caso de uso del histórico de datos

In [ ]:
##Cargamos el archivo csv con los datos de simulación

df_datos_simulacion = pd.read_csv('datos_simulacion.csv')

eventos = df_datos_simulacion['Evento'].unique()

#Preparamos los datos externos para el caso de uso

indice = np.random.randint(len(eventos))
caso_de_uso = df_datos_simulacion[df_datos_simulacion['Evento'] == eventos[indice]]

datos_externos = DatosExternos(datetime.datetime.strptime(str(caso_de_uso['Hora_inicio'].iloc[0]), '%Y-%m-%d %H:%M:%S'),
                               datetime.datetime.strptime(str(caso_de_uso['Hora_evento'].iloc[0]), '%Y-%m-%d %H:%M:%S'),
                               caso_de_uso['T_interior_inicial'],
                              23.5)

datos_externos.intervalos = 12
datos_externos.calcular_aforo_intervalos()
datos_externos.calcular_horas_intervalos()
datos_externos.temperatura_inicial = df_datos_simulacion["T_interior_inicial"].iloc[0]
datos_externos.temperatura_objetivo = 23.5
temperaturas = caso_de_uso["T_exterior"].iloc[0].split(' ')
t_aux = []
for x in temperaturas:
    if(len(x) > 2):
        x_aux = x.split('\n')[0]
        t_aux.append(x_aux)
        
temperaturas = []
for i in range(0, 12):
    temperaturas.append(float(t_aux[i]))
datos_externos.temperaturas_intervalos = temperaturas


optimizador = HVAcOptimizer()

optimizador.configurar_optimizador_simulacion(datos_externos = datos_externos,
                                 nombre_algoritmo="MOGA", 
                                 max_evaluaciones = 2000, poblacion_maxima = 200)

In [ ]:
optimizador.ejecutar_algoritmo()
resultados= optimizador.obtener_soluciones()
resultados['Fitness'] = calcular_fitness(resultados[['Confort', 'Consumo', 'COP', 'Vt', 'Tshow - Tend']])
resultados = resultados.sort_values(by=['Fitness'])
for i in range(12):
    if resultados["Programa"].iloc[0]["Hora"].iloc[0] == optimizador.datos_externos.horas_intervalos[i]:
        intervalo = i

In [ ]:
print(caso_de_uso[["Evento", "Hora_evento"]].iloc[0])
print("Temperatura objetivo: 23.5")

In [ ]:
resultados['Programa'].iloc[0]

In [ ]:
print(caso_de_uso['Programa'].iloc[0])

In [ ]:
consumo = np.fromstring(caso_de_uso["Consumos"].iloc[0][1:-1], sep = " ")
print("Valores objetivos optimizador: ")
print("Confort: ", round(resultados['Confort'].iloc[0], 2), 'ºC')
print("Temperatura final: ", round(resultados['Temperatura final'].iloc[0], 2), 'ºC')
print("Consumo: ", round(resultados["Consumo"].iloc[0], 2), 'KW')
print("Rendimiento medio: ", round(resultados["COP"].iloc[0], 2))
print("\nValores objetivos reales: ")
print("Confort: ", round(caso_de_uso['Confort'].iloc[0], 2), 'ºC')
print("Temperatura final: ", round(caso_de_uso['Confort'].iloc[0], 2) + 23.5, 'ºC')
print("Consumo: ", round(sum(consumo[intervalo:]), 2), 'KW')
print("Rendimiento medio: ", round(caso_de_uso["COP"].iloc[0], 2))

reales.append([caso_de_uso["Confort"].iloc[0], sum(consumo[intervalo:]), caso_de_uso["COP"].iloc[0]])
optimizados.append([resultados["Confort"].iloc[0], resultados["Consumo"].iloc[0], resultados["COP"].iloc[0]])

## Configuración para caso de uso real

In [ ]:
#Analizar numero de variables con nsga ii y spgea para asignar numero de variables en mopso
#Enviar paper de minimización jmetal
optimizador = HVAcOptimizer()

#Parámetros datetime: año, mes, día, hora, minutos
fecha_inicio = datetime.datetime(2020, 6, 30, 19, 30) 
fecha_evento = datetime.datetime(2020, 6, 30, 22, 30)

optimizador.configurar_optimizador(fecha_inicio = fecha_inicio, 
                                 fecha_evento = fecha_evento, 
                                 temperatura_inicial = 18,
                                 temperatura_objetivo = 24,
                                 nombre_algoritmo="MOGA", 
                                 max_evaluaciones = 10000, poblacion_maxima = 200)
optimizador.ejecutar_algoritmo()
resultados= optimizador.obtener_soluciones()
resultados['Fitness'] = calcular_fitness(resultados[['Confort', 'Consumo', 'COP', 'Vt', 'Tshow - Tend']])
resultados = resultados.sort_values(by=['Fitness'])

In [ ]:
print("Valores objetivos: \n")
print("Confort: ", round(resultados['Confort'].iloc[0], 2), 'ºC')
print("Temperatura final: ", round(resultados['Temperatura final'].iloc[0], 2), 'ºC')
print("Consumo: ", round(resultados["Consumo"].iloc[0], 2), 'KW')
print("Rendimiento medio: ", round(resultados["COP"].iloc[0], 2))
print("\nPrograma de climatización")
resultados['Programa'].iloc[0]

In [ ]:
fig = plt.figure(figsize=(15,10))
resultados.loc[resultados["COP"] > 50, "COP"] = 50
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel("Confort")
ax.set_ylabel("Consumo")
ax.set_zlabel("COP")

ax.scatter(resultados["Confort"],
               resultados["Consumo"],
               resultados["COP"])
ax.relim()
ax.autoscale_view(True, True)
ax.view_init(elev=30.0, azim=15.0)
ax.locator_params(nbins=4)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(111)
ax.set_xlabel("Temperatura final")
ax.set_ylabel("Consumo")
for i in range(len(resultados)):
    ax.scatter(resultados["Temperatura final"].iloc[i], resultados["Consumo"].iloc[i])
ax.autoscale_view(True, True)
plt.show()